## Celebritiy Rekcognition

### Goal
The goal of the below case study, is to create an application that detects celebrieties in an image and stores the name of the first celebrity a in database for future purposes.

Below are the sequence of steps that you will be doing:

#### Preparatory Phase:

1. Use CLI to create
    - EC2 instance
    - RDS instance
    - S3 bucket
2. Create appropriate role for each service used in the application
3. Create an EC2 instance with Jupyter notebook installed


#### Implementation phase:

1. Upload an image in the S3 bucket
2. Create a Lambda function that calls Amazon Rekognition service to detect the celebrities in the image.
3. Invoke the Lambda function to extract the celebrity in the image.
4. Store the file name and celebrity name in RDS database
6. Retrieve the list of all images and details from the database and display in the notebook.
 

### Implementation Phase

#### Image Upload 

In [ ]:
# installing the boto3 package
!pip3 install boto3 --user

In [ ]:
# importing boto3 library to access AWS services in Python
import boto3

In [ ]:
# Specifying the attributes for file upload
file_name = 'Jeff_Bezos.jpeg'

In [ ]:
# initiating the S3 client
s3 = boto3.client('s3')

In [ ]:
# Retrieve the list of existing buckets
response = s3.list_buckets()

In [ ]:
# List the bucket names
for bucket in response['Buckets']:
    print(bucket['name'])

In [ ]:
# Uploading the file on S3
bucket = 'testbuck-3'
object_name = 'Jeff_Bezos.jpeg'
s3.upload_file(file_name, bucket, object_name)

In [ ]:
# Retrieving the list of objects

response = s3.list_objects(Bucket=bucket)
for object in response['Contents']:
    print(object['Key'])

#### Lamda function - Creation

#Creating the handler file in the folder lambda (to be performed in separate file)

In [ ]:
# Installing boto3 in a target folder

!pip3 install boto3 --target ./lambda/boto3 boto3

In [ ]:
# Zipping the files together

%cd lambda
!zip -r9 ../celebrity_detector.zip .

In [ ]:
# Creating the function

!aws lambda create-function \
--function-name celebrity_detector \
--runtime python3.7 \
--zip-file fileb://celebrity_detector.zip \
--handler handler.lambda_handler \
--role arn:aws:iam::549124685073:role/csdrole \
--region us-east-1

#### Lamda function - Invoke AWS lambda

In [ ]:
# Invoking the function

import json
lam = boto3.client('lambda',region_name='us-east-1')

response = lam.invoke(FunctionName = 'celebrity_detector',
                      InvocationType = 'RequestResponse',
                      LogType = 'Tail',
                      Payload = json.dumps({"bucket":bucket,"object_name":object_name}))

In [ ]:
# Storing and printing the results in a variable

celebrity_name = response['Payload'].read().decode('utf8')
print(celebrity_name)

#### Database Operations

In [ ]:
# Installing the connector

!pip3 install mysql-connector --user

In [ ]:
# Importing the library

import mysql.connector

In [ ]:
# Connector object

dbc = mysql.connector.connect(
    host = "test-instance.cpt4oatcfnxe.us-east-1.rds.amazonaws.com",
    user = "master",
    passwd = "casestudy",
    port = 3306,
    db = "celebrities"
)

In [ ]:
# Defining the cursor object

cur = dbc.cursor()

In [ ]:
# Creating the table

cur.execute("CREATE TABLE celebrities (image VARCHAR(255), celebrity VARCHAR(255))")

In [ ]:
# Inserting the values into columns

sql = "INSERT INTO celebrities (image, celebrity) VALUES (%s, %s)"
val = (file_name, celebrity_name)
cur.execute(sql, val)

In [ ]:
# Read the stored values from RDS

cur.execute("SELECT image, celebrities FROM celebrities")
result = cur.fetchall()

In [ ]:
# Displaying the results

print(result)